# Load Dataset
Loading the dataset.json file into a pandas DataFrame for analysis.

In [1]:
import pandas as pd
import json

In [2]:
# Load the dataset.json file
with open('dataset.json', 'r') as file:
    data = json.load(file)

# Convert to DataFrame
df = pd.DataFrame(data)

In [3]:
# Flatten the data and extract required properties
flattened_data = []

for record in data:
    if 'trackDetails' in record:
        for track_detail in record['trackDetails']:
            # Initialize row data
            row = {}
            
            # Extract tracking number
            row['tracking_number'] = track_detail.get('trackingNumber', '')
            
            # Extract payment type (check for COD in special handlings)
            payment_type = 'Prepaid'  # Default
            if 'specialHandlings' in track_detail:
                for special_handling in track_detail['specialHandlings']:
                    if special_handling.get('type') == 'COD':
                        payment_type = 'COD'
                        break
            row['payment_type'] = payment_type
            
            # Extract shipment weight (combine value and units)
            if 'shipmentWeight' in track_detail:
                weight_value = track_detail['shipmentWeight'].get('value', '')
                weight_units = track_detail['shipmentWeight'].get('units', '')
                row['shipment_weight'] = f"{weight_value} {weight_units}" if weight_value and weight_units else ''
            else:
                row['shipment_weight'] = ''
            
            # Extract dates from datesOrTimes array
            pickup_datetime = ''
            delivery_datetime = ''
            
            if 'datesOrTimes' in track_detail:
                for date_time in track_detail['datesOrTimes']:
                    if date_time['type'] == 'ACTUAL_PICKUP':
                        pickup_datetime = date_time.get('dateOrTimestamp', '')
                    elif date_time['type'] == 'ACTUAL_DELIVERY':
                        delivery_datetime = date_time.get('dateOrTimestamp', '')
            
            row['pickup_datetime'] = pickup_datetime
            row['delivery_datetime'] = delivery_datetime
            
            # Extract Out for Delivery datetime(s) from events
            out_for_delivery_datetimes = []
            if 'events' in track_detail:
                for event in track_detail['events']:
                    if event.get('eventType') == 'OD' or 'delivery' in event.get('eventDescription', '').lower():
                        if 'timestamp' in event:
                            # Convert timestamp from milliseconds to readable format
                            timestamp_ms = int(event['timestamp'].get('$numberLong', 0))
                            import datetime
                            timestamp = datetime.datetime.fromtimestamp(timestamp_ms / 1000)
                            out_for_delivery_datetimes.append(timestamp.strftime('%Y-%m-%d %H:%M:%S'))
            
            row['out_for_delivery_datetimes'] = ', '.join(out_for_delivery_datetimes)
            
            # Extract pickup address (shipper address)
            if 'shipperAddress' in track_detail:
                shipper_addr = track_detail['shipperAddress']
                row['pickup_city'] = shipper_addr.get('city', '')
                row['pickup_state'] = shipper_addr.get('stateOrProvinceCode', '')
                # Try to find pickup pincode from events where pickup occurred
                pickup_pincode = ''
                if 'events' in track_detail:
                    for event in track_detail['events']:
                        if 'PU' in event.get('eventType', '') or 'pickup' in event.get('eventDescription', '').lower():
                            if 'address' in event and 'postalCode' in event['address']:
                                pickup_pincode = event['address']['postalCode']
                                break
                row['pickup_pincode'] = pickup_pincode
            else:
                row['pickup_pincode'] = ''
                row['pickup_city'] = ''
                row['pickup_state'] = ''
            
            # Extract drop address (destination address)
            if 'destinationAddress' in track_detail:
                dest_addr = track_detail['destinationAddress']
                row['drop_city'] = dest_addr.get('city', '')
                row['drop_state'] = dest_addr.get('stateOrProvinceCode', '')
                # Try to find drop pincode from delivery events
                drop_pincode = ''
                if 'events' in track_detail:
                    for event in track_detail['events']:
                        if event.get('eventType') == 'DL' or 'delivered' in event.get('eventDescription', '').lower():
                            if 'address' in event and 'postalCode' in event['address']:
                                drop_pincode = event['address']['postalCode']
                                break
                row['drop_pincode'] = drop_pincode
            else:
                row['drop_pincode'] = ''
                row['drop_city'] = ''
                row['drop_state'] = ''
            
            flattened_data.append(row)

# Convert to DataFrame
flattened_df = pd.DataFrame(flattened_data)

print(f"Flattened dataset shape: {flattened_df.shape}")
print(f"\nColumns: {list(flattened_df.columns)}")
flattened_df.head()

Flattened dataset shape: (99, 12)

Columns: ['tracking_number', 'payment_type', 'shipment_weight', 'pickup_datetime', 'delivery_datetime', 'out_for_delivery_datetimes', 'pickup_city', 'pickup_state', 'pickup_pincode', 'drop_city', 'drop_state', 'drop_pincode']


,tracking_number,payment_type,shipment_weight,pickup_datetime,delivery_datetime,out_for_delivery_datetimes,pickup_city,pickup_state,pickup_pincode,drop_city,drop_state,drop_pincode
0,391128701026,COD,14 KG,2020-03-16T15:44:00+05:30,2020-03-20T13:37:00+05:30,2020-03-20 10:16:00,Bangalore,KA,560048,Gurgaon,HR,122001
1,390901883808,Prepaid,14 KG,2020-03-06T16:07:00+05:30,2020-03-09T19:50:00+05:30,"2020-03-09 10:50:00, 2020-03-09 10:25:00",Bangalore,KA,560048,Bangalore,KA,560034
2,391128749178,Prepaid,14 KG,2020-03-16T15:44:00+05:30,2020-03-19T15:29:00+05:30,2020-03-19 12:30:00,Bangalore,KA,560048,Ahmedabad,GJ,380028
3,390807986805,Prepaid,14 KG,2020-03-03T16:19:00+05:30,2020-03-07T14:24:00+05:30,2020-03-07 10:27:00,Bangalore,KA,560048,New Delhi,DL,110088
4,390948921190,COD,14 KG,2020-03-09T15:12:00+05:30,2020-03-13T14:44:00+05:30,2020-03-13 09:32:00,Bangalore,KA,560048,Delhi,DL,110009


In [4]:
# Transform the data and compute inferred fields
import datetime
import pytz
from dateutil import parser

# Create a copy of the flattened dataframe for transformations
transformed_df = flattened_df.copy()

# Function to convert datetime to IST readable format
def convert_to_ist_readable(datetime_str):
    if not datetime_str or datetime_str == '':
        return ''
    try:
        # Parse the datetime string
        dt = parser.parse(datetime_str)
        # Convert to IST if not already
        if dt.tzinfo is None:
            # Assume UTC if no timezone info
            dt = dt.replace(tzinfo=pytz.UTC)
        ist = pytz.timezone('Asia/Kolkata')
        dt_ist = dt.astimezone(ist)
        return dt_ist.strftime('%Y-%m-%d %H:%M:%S IST')
    except:
        return datetime_str

# Function to calculate days between two dates
def calculate_journey_days(pickup_dt, delivery_dt):
    if not pickup_dt or not delivery_dt or pickup_dt == '' or delivery_dt == '':
        return ''
    try:
        pickup = parser.parse(pickup_dt)
        delivery = parser.parse(delivery_dt)
        # Calculate difference in days
        diff = delivery - pickup
        return round(diff.total_seconds() / (24 * 3600), 2)  # Convert to days with 2 decimal places
    except:
        return ''

# Function to count delivery attempts
def count_delivery_attempts(out_for_delivery_str, delivery_dt):
    if not out_for_delivery_str or out_for_delivery_str == '':
        return 1 if delivery_dt and delivery_dt != '' else 0
    
    try:
        # Split multiple out for delivery dates
        ofd_dates = [date.strip() for date in out_for_delivery_str.split(',') if date.strip()]
        
        if not delivery_dt or delivery_dt == '':
            return len(ofd_dates)
        
        # Parse delivery date
        delivery_date = parser.parse(delivery_dt).date()
        
        # Count unique dates for out for delivery
        unique_ofd_dates = set()
        for ofd_date_str in ofd_dates:
            try:
                ofd_date = parser.parse(ofd_date_str).date()
                unique_ofd_dates.add(ofd_date)
            except:
                continue
        
        # If delivery happened on same day as out for delivery, don't double count
        if delivery_date in unique_ofd_dates:
            return len(unique_ofd_dates)  # Just count the unique out for delivery dates
        else:
            return len(unique_ofd_dates) + 1  # Add 1 for final delivery
            
    except:
        return 1 if delivery_dt and delivery_dt != '' else 0

# Apply transformations
print("Transforming datetime fields to IST readable format...")
transformed_df['pickup_datetime_ist'] = transformed_df['pickup_datetime'].apply(convert_to_ist_readable)
transformed_df['delivery_datetime_ist'] = transformed_df['delivery_datetime'].apply(convert_to_ist_readable)

print("Calculating journey completion days...")
transformed_df['journey_days'] = transformed_df.apply(
    lambda row: calculate_journey_days(row['pickup_datetime'], row['delivery_datetime']), 
    axis=1
)

print("Calculating delivery attempts...")
transformed_df['delivery_attempts'] = transformed_df.apply(
    lambda row: count_delivery_attempts(row['out_for_delivery_datetimes'], row['delivery_datetime']), 
    axis=1
)

# Remove original datetime columns and replace with IST versions
print("Replacing original datetime columns with IST format...")
transformed_df = transformed_df.drop(['pickup_datetime', 'delivery_datetime'], axis=1)
transformed_df = transformed_df.rename(columns={
    'pickup_datetime_ist': 'pickup_datetime',
    'delivery_datetime_ist': 'delivery_datetime'
})

print(f"\nTransformed dataset shape: {transformed_df.shape}")
print(f"Columns: {list(transformed_df.columns)}")
transformed_df.head()


Transforming datetime fields to IST readable format...
Calculating journey completion days...
Calculating journey completion days...
Calculating delivery attempts...
Replacing original datetime columns with IST format...

Transformed dataset shape: (99, 14)
Columns: ['tracking_number', 'payment_type', 'shipment_weight', 'out_for_delivery_datetimes', 'pickup_city', 'pickup_state', 'pickup_pincode', 'drop_city', 'drop_state', 'drop_pincode', 'pickup_datetime', 'delivery_datetime', 'journey_days', 'delivery_attempts']
Calculating delivery attempts...
Replacing original datetime columns with IST format...

Transformed dataset shape: (99, 14)
Columns: ['tracking_number', 'payment_type', 'shipment_weight', 'out_for_delivery_datetimes', 'pickup_city', 'pickup_state', 'pickup_pincode', 'drop_city', 'drop_state', 'drop_pincode', 'pickup_datetime', 'delivery_datetime', 'journey_days', 'delivery_attempts']


,tracking_number,payment_type,shipment_weight,out_for_delivery_datetimes,pickup_city,pickup_state,pickup_pincode,drop_city,drop_state,drop_pincode,pickup_datetime,delivery_datetime,journey_days,delivery_attempts
0,391128701026,COD,14 KG,2020-03-20 10:16:00,Bangalore,KA,560048,Gurgaon,HR,122001,2020-03-16 15:44:00 IST,2020-03-20 13:37:00 IST,3.91,1
1,390901883808,Prepaid,14 KG,"2020-03-09 10:50:00, 2020-03-09 10:25:00",Bangalore,KA,560048,Bangalore,KA,560034,2020-03-06 16:07:00 IST,2020-03-09 19:50:00 IST,3.15,1
2,391128749178,Prepaid,14 KG,2020-03-19 12:30:00,Bangalore,KA,560048,Ahmedabad,GJ,380028,2020-03-16 15:44:00 IST,2020-03-19 15:29:00 IST,2.99,1
3,390807986805,Prepaid,14 KG,2020-03-07 10:27:00,Bangalore,KA,560048,New Delhi,DL,110088,2020-03-03 16:19:00 IST,2020-03-07 14:24:00 IST,3.92,1
4,390948921190,COD,14 KG,2020-03-13 09:32:00,Bangalore,KA,560048,Delhi,DL,110009,2020-03-09 15:12:00 IST,2020-03-13 14:44:00 IST,3.98,1


In [5]:
transformed_df.head()

,tracking_number,payment_type,shipment_weight,out_for_delivery_datetimes,pickup_city,pickup_state,pickup_pincode,drop_city,drop_state,drop_pincode,pickup_datetime,delivery_datetime,journey_days,delivery_attempts
0,391128701026,COD,14 KG,2020-03-20 10:16:00,Bangalore,KA,560048,Gurgaon,HR,122001,2020-03-16 15:44:00 IST,2020-03-20 13:37:00 IST,3.91,1
1,390901883808,Prepaid,14 KG,"2020-03-09 10:50:00, 2020-03-09 10:25:00",Bangalore,KA,560048,Bangalore,KA,560034,2020-03-06 16:07:00 IST,2020-03-09 19:50:00 IST,3.15,1
2,391128749178,Prepaid,14 KG,2020-03-19 12:30:00,Bangalore,KA,560048,Ahmedabad,GJ,380028,2020-03-16 15:44:00 IST,2020-03-19 15:29:00 IST,2.99,1
3,390807986805,Prepaid,14 KG,2020-03-07 10:27:00,Bangalore,KA,560048,New Delhi,DL,110088,2020-03-03 16:19:00 IST,2020-03-07 14:24:00 IST,3.92,1
4,390948921190,COD,14 KG,2020-03-13 09:32:00,Bangalore,KA,560048,Delhi,DL,110009,2020-03-09 15:12:00 IST,2020-03-13 14:44:00 IST,3.98,1
